In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-11"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9586,2024-10-11,Eua Nba,20:00,Minnesota Timberwolves,Philadelphia 76ers,1.65,2.22,222.5,1.87,1.91,-4.5,2.00,1.72,jeW0NsVM,0.606061,0.450450,0.534759,0.523560,0.056511,0.8,1.0,NaN,NaN,187.006,42.137842,0.225329,213.766,39.314571,0.183914,172.906,223.634,203.55,167.89,0.0,0.0,0.0,0.0,0.208295,0.014965,0.106446,-1.45,-0.290,-2.241379,0.604217,0.4,-0.204217,3.73,0.746,1.635389,0.457380,0.7,0.242620
9587,2024-10-11,Eua Nba,20:00,Washington Wizards,Toronto Raptors,2.22,1.65,222.5,1.89,1.89,1.5,2.00,1.72,pStcZsPA,0.450450,0.606061,0.529101,0.529101,0.056511,0.6,0.4,NaN,NaN,216.538,88.383473,0.408166,222.396,150.854047,0.678313,202.986,243.096,168.26,194.02,0.0,0.0,0.0,0.0,0.208295,0.000000,0.106446,-1.50,-0.300,-4.066667,0.394315,0.2,-0.194315,-5.00,-1.000,-0.650000,0.416579,0.1,-0.316579
9588,2024-10-11,Eua Nba,23:00,Golden State Warriors,Sacramento Kings,1.74,2.06,221.5,1.89,1.89,-3.5,2.05,1.68,YZPJw3F3,0.574713,0.485437,0.529101,0.529101,0.060150,0.2,0.6,NaN,NaN,232.576,64.165314,0.275890,165.522,26.262148,0.158663,176.592,206.086,157.38,165.90,0.0,0.0,0.0,0.0,0.119092,0.000000,0.140284,-0.40,-0.080,-9.250000,0.617808,0.5,-0.117808,-5.00,-1.000,-1.060000,0.455501,0.3,-0.155501
9589,2024-10-11,Eua Nba,23:00,Phoenix Suns,Detroit Pistons,1.60,2.32,218.5,1.88,1.90,-4.5,1.96,1.75,0MRBuPpi,0.625000,0.431034,0.531915,0.526316,0.056034,0.6,0.2,NaN,NaN,203.070,53.856812,0.265213,161.132,18.359465,0.113941,181.936,435.642,187.45,153.99,0.0,0.0,0.0,0.0,0.259754,0.007483,0.080050,-1.71,-0.342,-1.754386,0.648493,0.6,-0.048493,-2.96,-0.592,-2.229730,0.262689,0.4,0.137311
9590,2024-10-11,Eua Nba,23:30,Los Angeles Clippers,Portland Trail Blazers,1.65,2.22,216.5,1.89,1.89,-4.5,2.00,1.72,U1JSyspG,0.606061,0.450450,0.529101,0.529101,0.056511,0.4,0.2,NaN,NaN,214.020,92.954638,0.434327,204.168,64.826852,0.317517,178.122,298.388,162.15,140.08,0.0,0.0,0.0,0.0,0.208295,0.000000,0.106446,0.08,0.016,40.625000,0.538630,0.4,-0.138630,1.77,0.354,3.446328,0.317724,0.4,0.082276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9654,2024-10-11,Rússia Premier League Feminina,11:00,Neftyanik Omsk F,Dynamo Moscow F,1.61,2.22,133.5,1.85,1.85,-4.5,2.00,1.69,buhQE4EQ,0.621118,0.450450,0.540541,0.540541,0.071568,0.0,0.6,NaN,NaN,497.938,559.447490,1.123528,838.612,681.258933,0.812365,0.000,1011.708,475.60,1200.00,1.0,1.0,0.0,0.0,0.225240,0.000000,0.118809,0.00,0.000,inf,0.000000,0.0,0.000000,-5.00,-1.000,-1.220000,0.000000,0.0,0.000000
9655,2024-10-11,Rússia Premier League Feminina,13:00,Dynamo Kursk F,Nadezhda F,1.22,3.98,137.5,1.85,1.85,-11.5,2.03,1.60,dtwNd5hr,0.819672,0.251256,0.540541,0.540541,0.070928,0.8,0.0,NaN,NaN,107.940,32.829352,0.304144,123.466,49.031757,0.397128,107.048,0.000,89.89,115.42,0.0,0.0,0.0,0.0,0.750621,0.000000,0.167524,1.28,0.256,0.859375,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9656,2024-10-11,Rússia Premier League Feminina,13:00,MBA Moscow F,Samara F,1.35,3.02,137.5,1.85,1.85,-8.5,2.05,1.59,46tVfqNf,0.740741,0.331126,0.540541,0.540541,0.071867,0.0,0.6,NaN,NaN,126.306,49.051394,0.388354,220.974,171.469246,0.775970,0.000,280.328,86.32,112.42,0.0,0.0,0.0,0.0,0.540443,0.000000,0.178719,0.00,0.000,inf,0.000000,0.0,0.000000,3.51,0.702,2.877493,0.000000,0.0,0.000000
9657,2024-10-11,Sérvia 1. Zls Femin

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
